In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def load_demes(path_to_data):
    """
    Load the data from the specified path, read the csv file, 
    and convert the 'AverageArray' column into lists of floats.

    Parameters:
    path_to_data (str): The path to the data file.

    Returns:
    pandas.DataFrame: DataFrame with 'AverageArray' column converted to lists of floats.
    """

    # Read the CSV file
    df = pd.read_csv(path_to_data, sep='\t', header=0)

    # Convert 'AverageArray' column into lists of floats
    df['AverageArray'] = df['AverageArray'].apply(lambda x: np.fromstring(x, sep=' '))

    return df


In [ ]:
def separate_by_generation(df):
    """
    Separate the DataFrame by generation and return a dictionary 
    with generations as keys and corresponding DataFrames as values.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.

    Returns:
    dict: Dictionary with generations as keys and DataFrames as values.
    """
    generation_dict = {}
    for generation in df['Generation'].unique():
        generation_dict[generation] = df[df['Generation'] == generation]
    
    return generation_dict


In [ ]:
example_number = '2'
path_to_data = 'eg' + example_number + '/final_demes.txt'
df = load_demes(path_to_data)


In [ ]:
gen_dict = separate_by_generation(df)


In [ ]:
def plot_for_generation(generation_dict, generation):
    """
    Generate plots for a specified generation. This includes histograms for each deme, 
    scatter plots above the diagonal, and 2D histograms below the diagonal in the grid.

    Parameters:
    generation_dict (dict): Dictionary with generations as keys and DataFrames as values.
    generation (float): The specific generation to plot.
    """
    df = generation_dict[generation]

    # Separate by side
    for side in df['Side'].unique():
        side_df = df[df['Side'] == side]

        # Get unique demes
        demes = side_df['Deme'].unique()
        num_demes = len(demes)

        # Create a grid of plots
        fig, axs = plt.subplots(num_demes, num_demes, figsize=(15, 15))

        # Adjust for cases with a single deme
        if num_demes == 1:
            axs = np.array([[axs]])

        # Iterate over each pair of demes
        for i in range(num_demes):
            for j in range(num_demes):
                ax = axs[i, j]

                if i == j:  # Diagonal: Histograms
                    deme_df = side_df[side_df['Deme'] == demes[i]]
                    data = deme_df.drop(columns=['Generation', 'Deme', 'Side', 'Population']).values.flatten()
                    ax.hist(data, bins=30, color='skyblue', edgecolor='black')
                    ax.set_title(f'Gen {generation}, Side {side}, Deme {demes[i]}')
                elif i < j:  # Upper Triangle: Scatter plots
                    deme_df_i = side_df[side_df['Deme'] == demes[i]]
                    deme_df_j = side_df[side_df['Deme'] == demes[j]]
                    ax.scatter(deme_df_i.iloc[:, 4:].values.flatten(), deme_df_j.iloc[:, 4:].values.flatten(), alpha=0.5)
                    ax.set_title(f'Scatter: Deme {demes[i]} vs Deme {demes[j]}')
                else:       # Lower Triangle: 2D Histograms
                    deme_df_i = side_df[side_df['Deme'] == demes[i]]
                    deme_df_j = side_df[side_df['Deme'] == demes[j]]
                    ax.hist2d(deme_df_i.iloc[:, 4:].values.flatten(), deme_df_j.iloc[:, 4:].values.flatten(), bins=30, cmap='Blues')
                    ax.set_title(f'Hist2D: Deme {demes[i]} vs Deme {demes[j]}')

        plt.tight_layout()
        plt.show()


In [ ]:
def create_deme_side_dataframe(df):
    """
    Create a new DataFrame where columns are grouped by side, 
    and each column corresponds to a unique deme within each side.
    Each row in these columns contains one value from the respective 'AverageArray'.

    Parameters:
    df (pandas.DataFrame): The input DataFrame with 'AverageArray' as lists.

    Returns:
    pandas.DataFrame: New DataFrame with separate columns for each deme, grouped by side.
    """
    # Create a dictionary to hold the data, grouped by side
    data = {}

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        # Create a key for each unique combination of deme and side
        side = row['Side']
        deme_key = f"{row['Deme']}"

        # Initialize the side group if not already present
        if side not in data:
            data[side] = {}

        # Append the data to the corresponding key in the side group
        if deme_key not in data[side]:
            data[side][deme_key] = []
        data[side][deme_key].extend(row['AverageArray'])

    # Convert the dictionary to a DataFrame, with columns grouped by side
    reshaped_data = {}
    for side, demes in data.items():
        for deme, values in demes.items():
            column_name = f"{side}_{deme}"
            reshaped_data[column_name] = pd.Series(values)

    reshaped_df = pd.DataFrame(reshaped_data)

    return reshaped_df


In [ ]:
gen_dict.keys()

In [ ]:
new_df = create_deme_side_dataframe(gen_dict[60])

In [ ]:
def plot_deme_side_data(df):
    # Get the number of demes (columns)
    demes = df.columns
    num_demes = len(demes)

    # Create a grid of plots
    fig, axs = plt.subplots(num_demes, num_demes, figsize=(15, 15))

    # Adjust for cases with a single deme
    if num_demes == 1:
        axs = np.array([[axs]])

    # Iterate over each pair of demes
    for i in range(num_demes):
        for j in range(num_demes):
            ax = axs[i, j]

            if i == j:  # Diagonal: Histograms for individual demes
                ax.hist(df[demes[i]].dropna(), bins=30, color='skyblue', edgecolor='black')
                ax.set_title(f'Histogram: {demes[i]}')
            elif i < j:  # Upper Triangle: Scatter plots
                ax.scatter(df[demes[i]], df[demes[j]], alpha=0.5)
                ax.set_title(f'Scatter: {demes[i]} vs {demes[j]}')
            else:       # Lower Triangle: 2D Histograms
                ax.hist2d(df[demes[i]].dropna(), df[demes[j]].dropna(), bins=30, cmap='Blues')
                ax.set_title(f'Hist2D: {demes[i]} vs {demes[j]}')

            ax.set_xlabel(demes[i])
            ax.set_ylabel(demes[j])

    plt.tight_layout()
    plt.show()


In [ ]:
plot_deme_side_data(new_df)

In [ ]:
def gland_distance(gland1, gland2):
    """
    Calculate a hybrid Manhattan-like distance between two glands.

    :param gland1: Array-like, values for the first gland.
    :param gland2: Array-like, values for the second gland.
    :return: Float, the calculated hybrid Manhattan distance.
    """
    # Calculate absolute differences
    differences = np.abs(np.array(gland1) - np.array(gland2))

    # Define buckets and scale contributions
    # Adjust these thresholds and scales as needed
    buckets = [(0, 0.25, 0.1), (0.25, 0.5, 0.3), (0.5, 0.75, 0.7), (0.75, 1.0, 1.0)]
    
    # Initialize distance
    distance = 0
    # Apply bucket scaling
    for lower, upper, scale in buckets:
        mask = (differences >= lower) & (differences < upper)
        distance += np.sum(differences[mask] * scale)

    # Normalize by the number of sites
    total_distance = distance / len(differences)

    return total_distance


In [ ]:
def compute_distance_matrix(df):
    num_glands = df.shape[1]
    distance_matrix = np.zeros((num_glands, num_glands))

    # Compute the distance for each pair of glands
    for i in range(num_glands):
        for j in range(num_glands):
            distance_matrix[i, j] = gland_distance(df.iloc[:, i], df.iloc[:, j])

    return distance_matrix


In [ ]:
distance_matrix = compute_distance_matrix(new_df)

In [ ]:
# Plotting the heatmap with gland names as labels
plt.figure(figsize=(12, 10))
sns.heatmap(distance_matrix, annot=True, cmap='coolwarm', xticklabels=new_df.columns, yticklabels=new_df.columns)
plt.title("Heatmap of Distances Between Glands")
plt.xlabel("Gland")
plt.ylabel("Gland")
plt.show()
